In [1]:
import os, sys
sys.path.append('../')
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch
import torchvision
import torchvision.transforms as transforms
from datasets.coco import CocoDataset
from utils.transforms import ImageToTensor, CategoryToTensor, BoxToTensor, Resize, CocoBoxToFracBoundaryBox
from functools import partial

import torch.nn as nn
import torch.nn.functional as F

from model.detector import SSD300

from torch.utils.tensorboard import SummaryWriter

/Users/jcheng/opt/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
# transforms
IMG_SZ = 300
toTensor_tfms = [ImageToTensor(), 
                 CategoryToTensor(), 
                 BoxToTensor()]
resize_tfms   = [Resize((IMG_SZ, IMG_SZ)),
                 CocoBoxToFracBoundaryBox()]
tfms = transforms.Compose(toTensor_tfms + resize_tfms)

# create dataset & dataloader
ds = CocoDataset('../datasets/', 'samples', 'instances', transforms=tfms)
dl = torch.utils.data.DataLoader(ds, batch_size=4, shuffle=True, num_workers=2,
                                 collate_fn=partial(ds.collate_fn, img_resized=True))

# create tensorboard summary writer
writer = SummaryWriter('runs/vgg_base')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
# get some random training images
dataiter = iter(dl)
dbatch = dataiter.next()
print(dbatch.keys())

dict_keys(['images', 'segs', 'cats', 'boxes', 'dims'])


In [4]:
# create grid of images
img_grid = torchvision.utils.make_grid(dbatch['images'])

# write to tensorboard
writer.add_image('coco_samples', img_grid)

In [5]:
net = SSD300(n_classes=len(ds.id2cat.keys()))

In [6]:
writer.add_graph(net, dbatch['images'])
writer.close()

/Users/jcheng/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [7]:
%load_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 40645), started 2 days, 13:16:52 ago. (Use '!kill 40645' to kill it.)